# Capstone Project 1

## Title: Prediction of electricity rates (cost/Kwh) and consumption
### Author: Elizabeth Izarra

### Problem:
Electricity has a very dynamic market price since it is a commodity that is essential for daily life and 
non-storable where generation and demand must be continuously balanced. This is turn make it dependable of the 
weather conditions.    

### Data:

U.S. Energy Information Administration (EIA)

Data Sets:
- Net Generation
- Electricity - Sales (consumption), revenue, prices & customers
- Average cost of fossil fuels delivered by state
- Consumption of fuels used to generate electricity
- Electricity Stock price
 

### ---------------------------------------------------------------------------------------------------

In [1]:
# import the relevant modules
import requests
from pandas.io.json import json_normalize
import pandas as pd

In [2]:
def get_data_from_url(API_query):
    # Package the request, send the request and catch the response: r
    r=requests.get(API_query)

    # Inspect the JSON structure, and exploration of how nested it is,
    # as well as the overall structure

    # Decode the JSON data into a dictionary: json_data
    json_data=r.json()
    return json_data

### Exploring U.S. Energy Information Administration (EIA)

#### Selecting Variables of Interest

In [3]:
API_KEY='a32b693a24d623394f601a1c4262fb10'

In [4]:
#Exploring the available variables
categoryID='0'
#Call the EIA API and pull out the data categories
url='http://api.eia.gov/category/?api_key='+API_KEY+'&category_id='+categoryID
json_data=get_data_from_url(url)

In [5]:
#Getting the childcategories into a DataFrame
df_cat_IDs_0=json_normalize(json_data['category']['childcategories'])
df_cat_IDs_0

,category_id,name
0,1,Net generation
1,32,Total consumption (Btu)
2,33,Consumption for electricity generation (Btu)
3,34,Consumption for useful thermal output (Btu)
4,35,Total consumption
5,36,Consumption for electricity generation
6,37,Consumption for useful thermal output
7,38,Retail sales of electricity
8,39,Revenue from retail sales of electricity
9,40,Average retail price of electricity


#### Selecting and Retriving Series of interest

In [6]:
#Exploring series per categories

def get_series(idx, f, start, end):
       
  
        categoryID=str(df_cat_IDs_0.iloc[idx,0]) 
        variable=df_cat_IDs_0['name'][idx]

        #API for series name exploration
        url='http://api.eia.gov/category/?api_key='+API_KEY+'&category_id='+categoryID
        json_data=get_data_from_url(url)

        df_ser_ID=json_normalize(json_data['category']['childseries'])

        #API for series name exploration
        url='http://api.eia.gov/category/?api_key='+API_KEY+'&category_id='+categoryID
        json_data=get_data_from_url(url)

        df_ser_ID=json_normalize(json_data['category']['childseries'])

        #getting the series ID for the monthly reporting per state
        #print(f, df_ser_ID.keys())
        ser_IDs=df_ser_ID[df_ser_ID['f'].str.contains(f)]

        #print(ser_IDs.head(), categoryID, variable) 

        #Getting series

        for i in range(len(ser_IDs)):
            #Building API
            seriesID=str(ser_IDs.iloc[i,2]) 
            url='http://api.eia.gov/series/?series_id='+seriesID+'&api_key='+API_KEY+'&start='+start+'&end='+end
            #Retriving data from URL
            json_data=get_data_from_url(url)
            #Extracting information of interest
            h1=json_normalize(json_data,['series'])
            h=json_normalize(json_data,['series',['data']])
            #Adding names to columns
            h.columns=['date',variable]
            h['geography']=h1['geography']
            h['unit']=h1['units']
            #h['variable']=variable
            #Adding missing data
            h.fillna(method='ffill', inplace=True)
            seriesID, json_data, h
            if i==0:
                df_data_cat=h
            else:

                df_data_cat=df_data_cat.append(h, ignore_index=True, sort=False)
        
        return df_data_cat

In [9]:
#Selecting variables of interest
variables_idx=[0,7,8,9,18] #list of indexes of the variables of interest of df_cat_IDs_0
f='A' #Frequency can be 'A':Annual, 'Q':Quarter or 'M':Monthly 
start='2013'
end='2013'
c_flag=0
for idx in variables_idx:
    #categoryID=str(df_cat_IDs_0.iloc[idx,0]) 
    #variable=df_cat_IDs_0['name'][idx]

    ##API for series name exploration
    #url='http://api.eia.gov/category/?api_key='+API_KEY+'&category_id='+categoryID
    #json_data=get_data_from_url(url)

    #df_ser_ID=json_normalize(json_data['category']['childseries'])
    #print(idx, df_ser_ID)

    df_data_cat=get_series(idx, f, start, end)
    #print(df_data_cat)
    if c_flag==0:
            df_data=df_data_cat
            c_flag=1
    else:
            
            #Mergin df2 with dictionary Values{code: name} from previous exercise
            df_data=df_data.merge(df_data_cat, how='outer', left_on=['date','geography'], right_on=['date','geography'])

    #print(df_data)

In [10]:
df_data


,date,Net generation,geography,unit_x,Retail sales of electricity,unit_y,Revenue from retail sales of electricity,unit_x,Average retail price of electricity,unit_y,Number of customer accounts,unit
0,2013,6.496822e+03,USA-AK,thousand megawatthours,6.268212e+03,million kilowatthours,1033.34690,million dollars,16.49,cents per kilowatthour,3.277599e+05,number of customers
1,2013,1.505729e+05,USA-AL,thousand megawatthours,8.785211e+04,million kilowatthours,7923.66243,million dollars,9.02,cents per kilowatthour,2.524633e+06,number of customers
2,2013,6.032249e+04,USA-AR,thousand megawatthours,4.668303e+04,million kilowatthours,3703.71061,million dollars,7.93,cents per kilowatthour,1.557321e+06,number of customers
3,2013,1.133260e+05,USA-AZ,thousand megawatthours,7.566204e+04,million kilowatthours,7669.32240,million dollars,10.14,cents per kilowatthour,2.947070e+06,number of customers
4,2013,2.000771e+05,USA-CA,thousand megawatthours,2.613325e+05,million kilowatthours,37372.30791,million dollars,14.30,cents per kilowatthour,1.510754e+07,number of customers
5,2013,5.293744e+04,USA-CO,thousand megawatthours,5.344165e+04,million kilowatthours,5281.85441,million dollars,9.88,cents per kilowatthour,2.544333e+06,number of customers
6,2013,3.561079e+04,USA-CT,thousand megawatthours,2.982452e+04,million kilowatthours,4669.08413,million dollars,15.66,cents per kilowatthour,1.609953e+06,number of customers
7,2013,6.585208e+01,USA-DC,thousand megawatthours,1.108552e+04,million kilowatthours,1314.04368,million dollars,11.85,cents per kilowatthour,2.618542e+05,number of customers
8,2013,7.760861e+03,USA-DE,thousand megawatthours,1.134786e+04,million kilowatthours,1236.84343,million dollars,10.90,cents per kilowatthour,4.560401e+05,number of customers
9,2013,6.220732e+05,USA-IL+USA-IN+USA-MI+USA-OH+USA-WI,thousand megawatthours,5.697616e+05,million kilowatthours,53557.75012,million dollars,9.40,cents per kilowatthour,2.213383e+07,number of customers


In [11]:
df_data.groupby(by=['geography'])['date'].count()



geography
USA                                                               1
USA-AK                                                            1
USA-AK+USA-HI                                                     1
USA-AL                                                            1
USA-AL+USA-KY+USA-MS+USA-TN                                       1
USA-AR                                                            1
USA-AR+USA-LA+USA-OK+USA-TX                                       1
USA-AZ                                                            1
USA-AZ+USA-CO+USA-ID+USA-MT+USA-NM+USA-NV+USA-UT+USA-WY           1
USA-CA                                                            1
USA-CA+USA-OR+USA-WA                                              1
USA-CO                                                            1
USA-CT                                                            1
USA-CT+USA-MA+USA-ME+USA-NH+USA-RI+USA-VT                         1
USA-DC                                